<a href="https://colab.research.google.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/tutorialParetoOptimality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Optimization of Autonomous Bus Stops
## Game Theory and Multi-Objective Optimization


**Context:** An autonomous bus must determine its optimal stops in a city organized in a 10×10 grid. Ten passengers inside the bus have different preferences for the stops. What are the Pareto optima and the nash equilibria ?


In [5]:
#few libraries
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.spatial.distance import cityblock
import random
from typing import List, Tuple, Set
from dataclasses import dataclass

### Model
Model of the travellers and their preferences

In [20]:
@dataclass
class PointPreference:
    """ a preferred point and its value"""
    x: int
    y: int
    value: float
    
    def position(self) -> Tuple[int, int]:
        return (self.x, self.y)
    
    def __repr__(self):
        """
        string representation of a preferred point.
        """
        return f"PointPreference ({self.x},{self.y})={self.value:.2f}"

In [51]:
class Passenger:
    """
    Model of a traveler with his stop preferences
    """
    def __init__(self, id_passager: int, preferences: List[PointPreference]):
        self.id = id_passager
        # sort preferred points by value descending
        self.preferences = sorted(preferences, key=lambda p: p.value, reverse=True)
        
    def compute_point_utility(self, x: int, y: int, distance_max: float = 20) -> float:
        """
        Compute the utility of any point for this passenger. 
        The value decreases linearly with the distance to the nearest preferred point.
        
        Args:
            x, y: coordinates of the point to evaluate
            distance_max: distance at which the utility becomes zero
        """
        # is it a preferred point?
        for pref in self.preferences:
            if pref.x == x and pref.y == y:
                return pref.value
        
        # not a preferred point, compute utility based on distance to preferred points
        max_utility = 0
        for pref in self.preferences:
            #use cityblock (=manhattan, laplacian) distance because city is a grid
            dist = cityblock((x, y), (pref.x, pref.y))
            utility = max(0, pref.value * (1 - dist / distance_max))
            max_utility = max(max_utility, utility)
        
        return max_utility
    
    def compute_max_utility(self, stops: Set[Tuple[int, int]]) -> float:
        """
        Compute the max utility for a given set of stops.
        Takes the best available stop because, only 1 stop will be used.
        """
        # no stops provided -> utility is zero
        if not stops: return 0.0
        
        utilities = [self.compute_point_utility(x, y) for x, y in stops]
        return max(utilities)
    
    def compute_min_utility(self, stops: Set[Tuple[int, int]]) -> float:
        """
        Compute the min utility for a given set of stops.
        Takes the worst available stop because, only 1 stop will be used.
        """
        # no stops provided -> utility is zero
        if not stops: return 0.0
        
        utilities = [self.compute_point_utility(x, y) for x, y in stops]
        return min(utilities)

    def __repr__(self):
        """
        string representation of a passenger.
        """
        points_str = ', '.join([str(p) for p in self.preferences])
        return f"Passager {self.id} - Prefs=[{points_str}]"

In [48]:
# Example usage
p = Passenger(1,  [PointPreference(2, 2, 10), PointPreference(5, 5, 5)])
point = {"x": 2, "y": 2}
print("",point, " : utility =  ", p.compute_point_utility(point["x"], point["y"]))  
point = {"x": 3, "y": 3}
print("",point, " : utility =  ", p.compute_point_utility(point["x"], point["y"]))  
point = {"x": 5, "y": 4}
print("",point, " : utility =  ", p.compute_point_utility(point["x"], point["y"]))  
point = {"x": 5, "y": 5}
print("",point, " : utility =  ", p.compute_point_utility(point["x"], point["y"]))  
print("max utility for these points =  ", p.compute_max_utility({(2, 2), (3, 3), (5, 4), (5, 5)}))  
print("min utility for these points =  ", p.compute_min_utility({(2, 2), (3, 3), (5, 4), (5, 5)}))  


 {'x': 2, 'y': 2}  : utility =   10
 {'x': 3, 'y': 3}  : utility =   9.0
 {'x': 5, 'y': 4}  : utility =   7.5
 {'x': 5, 'y': 5}  : utility =   5
max utility for these points =   10
min utility for these points =   5


----
#### Data generation

Let's generate passengers..

In [36]:
def generate_random_passengers(nbp: int = 10, grid_size: int = 10,  seed: int = 42) -> List[Passenger]:
    """
    Generates np passengers with coherent random preferences.
    
    Args:
        np: nb of passengers to generate
        grid_size:  (10x10 par défaut)
        seed: seed for random generator (start value for the sequence of random numbers)
    """
    random.seed(seed)
    
    passengers = []
    
    for i in range(nbp):
        
        # generate 3 distinct preferred points
        chosen_points = set()
        preferences = []
        
        # generate 3 random values, sorted descending
        valeurs = sorted([random.uniform(50, 100), 
                         random.uniform(30, 70), 
                         random.uniform(10, 50)], reverse=True)
        
        for valeur in valeurs:
            # avoid choosing the same point twice (only 3 points among 100 possible, so the loop will end quickly)
            while True:
                x = random.randint(0, grid_size - 1)
                y = random.randint(0, grid_size - 1)
                if (x, y) not in chosen_points:
                    chosen_points.add((x, y))
                    break
            
            preferences.append(PointPreference(x, y, valeur))
        
        passengers.append(Passenger(i,  preferences))
    
    return passengers


In [37]:
# Example usage
passengers = generate_random_passengers(3, 10, 42)
for p in passengers: print(p)

Passager 0 - Prefs=[PointPreference (3,2)=81.97, PointPreference (1,8)=31.00, PointPreference (1,9)=21.00]
Passager 1 - Prefs=[PointPreference (8,9)=71.10, PointPreference (0,8)=31.19, PointPreference (3,8)=18.75]
Passager 2 - Prefs=[PointPreference (0,2)=70.98, PointPreference (6,5)=47.97, PointPreference (4,2)=21.13]


In [55]:
#function that can be used or not to compute max utilities of a set of stops for all passengers, 
def compute_max_utilities(stops: Set[Tuple[int, int]],  passengers: List[Passenger]) -> np.ndarray:
    """
    compute utilities vector for all passengers   
    Args:
        stops:  selected stops set 
        passengers:  passengers list
    Returns:
        Array of len(passengers) with  individual utilities
    """
    return np.array([p.compute_max_utility(stops) for p in passengers])

#function that can be used or not to compute min utilities of a set of stops for all passengers, 
def compute_min_utilities(stops: Set[Tuple[int, int]],  passengers: List[Passenger]) -> np.ndarray:
    """
    compute utilities vector for all passengers   
    Args:
        stops:  selected stops set 
        passengers:  passengers list
    Returns:
        Array of len(passengers) with  individual utilities
    """
    return np.array([p.compute_min_utility(stops) for p in passengers])


In [56]:
# Example usage
passengers = generate_random_passengers(5, 10, 42)
for p in passengers: print(p)
stops = {(2, 2), (5, 5), (7, 7)}
print("Proposed stops: ", stops)
max_utilities = compute_max_utilities(stops, passengers)
min_utilities = compute_min_utilities(stops, passengers)
print("Interests of the points for the passengers")
i = 0
for s in stops:
    print(f"{s} max utility: {max_utilities[i]}, min utility: {min_utilities[i]}")
    i+=1


Passager 0 - Prefs=[PointPreference (3,2)=81.97, PointPreference (1,8)=31.00, PointPreference (1,9)=21.00]
Passager 1 - Prefs=[PointPreference (8,9)=71.10, PointPreference (0,8)=31.19, PointPreference (3,8)=18.75]
Passager 2 - Prefs=[PointPreference (0,2)=70.98, PointPreference (6,5)=47.97, PointPreference (4,2)=21.13]
Passager 3 - Prefs=[PointPreference (6,1)=60.77, PointPreference (5,5)=60.54, PointPreference (9,4)=14.09]
Passager 4 - Prefs=[PointPreference (6,1)=90.36, PointPreference (8,4)=59.19, PointPreference (9,5)=31.45]
Proposed stops:  {(5, 5), (7, 7), (2, 2)}
Interests of the points for the passengers
(5, 5) max utility: 77.87277292674948, min utility: 45.084236957591806
(7, 7) max utility: 60.431677336624, min utility: 24.883631844492232
(2, 2) max utility: 63.87839194327464, min utility: 40.773107573651025


---
### Paretos points


To find the pareto points (front), we have to check all the possible deal points (10x10 here).
So each passenger has to valuate all of the 100 points.
Next, we have to find the list of Pareto optimal points (a pareto optimal point is not dominated by any other point).


Define the function that determinate if a point is a Pareto point or not.

N.B. *allStops* a set of set of 3 points *p* where the bus can stop.
Each passenger have their preference on this proposals.

Let *busStop* a set of 3 points.

- *busStop* is Pareto-optimal ⟺ ∄ *OtherStops* such as *OtherStops* dominates *busStop*
   -  *OtherStops* dominates *busStop* if (Ui​(*OtherStops*)≥Ui​(*busStop*) ∀i) and (Ui​(*OtherStops*)>Ui​(*busStop*) for at least one i)
   -  i = passenger i


In [ ]:
def is_pareto_optimal(stop: Tuple[int, int], all_points: List[Set[Tuple[int, int]]], passengers: List[Passenger]) -> bool:
    """"
    Determine if a given stop_set is Pareto optimal among all_sets for the given passengers.
    Args:
        stop: The stop to evaluate.
        all_points: List of all possible  stops.
        passengers: List of passengers.
    """
    is_pareto_optimal = False
    
    #TODO: implement the function
    return is_pareto_optimal

#### Tests 
Generate a list of set of 3 stop; and find the pareto optimal ones (if there exists any).

Display them and their utilities for the passengers.


In [ ]:
def get_pareto_optimal(all_points: List[Set[Tuple[int, int]]], passengers: List[Passenger]) -> List[Set[Tuple[int, int]]]:
    """"
    Compute Pareto optimal point among all_sets for the given passengers.
    Args:
        all_points: List of all possible  stops.
        passengers: List of passengers.
    """
    is_pareto_optimal = False
    
    #TODO: implement the function
    return pareto_points